Тема ВКР: "Система автоматизированного распределения инвестиций в акции компаний на основе финансовых ограничений и сроков инвестирования с использованием машинного обучения"

LANE - Learning Algorithm for Net Equity

In [1]:
#импорт библиотек
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
def getcsv(name, clear=False):
  df = pd.read_csv(name)
  df.columns = df.columns.str.replace('<', '').str.replace('>', '')
  if clear:
    return df[['OPEN', 'HIGH', 'LOW', 'VOL']]
  else:
    return df

In [9]:
#https://www.finam.ru/profile/moex-akcii/gazprom/export/
# df = pd.read_csv('GAZP_old.csv')
df = pd.read_csv('data/GAZP_220401_240206.csv')
df.columns = df.columns.str.replace('<', '').str.replace('>', '')
df.head()

,TICKER,PER,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL
0,GAZP,D,20220401,0,245.90,258.70,242.48,251.4,50423650
1,GAZP,D,20220404,0,254.04,259.99,237.31,252.9,40926330
2,GAZP,D,20220405,0,254.01,256.39,231.45,243.5,39850580
3,GAZP,D,20220406,0,238.73,247.80,234.41,239.7,20530890
4,GAZP,D,20220407,0,240.04,246.80,239.13,245.0,22367990


In [10]:
df.isna().sum()

TICKER    0
PER       0
DATE      0
TIME      0
OPEN      0
HIGH      0
LOW       0
CLOSE     0
VOL       0
dtype: int64

In [11]:
df.dtypes

TICKER     object
PER        object
DATE        int64
TIME        int64
OPEN      float64
HIGH      float64
LOW       float64
CLOSE     float64
VOL         int64
dtype: object

In [12]:
df['DATE'] = df['DATE'].astype(str)
df['DATE'] = pd.to_datetime(df['DATE'], format='%Y%m%d')
df.dtypes

TICKER            object
PER               object
DATE      datetime64[ns]
TIME               int64
OPEN             float64
HIGH             float64
LOW              float64
CLOSE            float64
VOL                int64
dtype: object

In [13]:
df

,TICKER,PER,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL
0,GAZP,D,2022-04-01,0,245.90,258.70,242.48,251.40,50423650
1,GAZP,D,2022-04-04,0,254.04,259.99,237.31,252.90,40926330
2,GAZP,D,2022-04-05,0,254.01,256.39,231.45,243.50,39850580
3,GAZP,D,2022-04-06,0,238.73,247.80,234.41,239.70,20530890
4,GAZP,D,2022-04-07,0,240.04,246.80,239.13,245.00,22367990
...,...,...,...,...,...,...,...,...,...
464,GAZP,D,2024-01-31,0,163.41,166.48,162.90,166.33,39738490
465,GAZP,D,2024-02-01,0,166.71,166.75,164.80,165.25,20836140
466,GAZP,D,2024-02-02,0,165.16,167.85,164.40,164.40,32725330
467,GAZP,D,2024-02-05,0,164.45,165.22,163.55,164.87,17938190


In [14]:
fig = go.Figure()
for name in df['TICKER'].unique():
    fig.add_trace(go.Scatter(x=df.loc[df['TICKER'] == name]['DATE'], y=df.loc[df['TICKER'] == name]['CLOSE'], mode='lines+markers',  name=name))
fig.update_layout(title=str(df['TICKER'].unique()), xaxis_title='Дата', yaxis_title='Цена')
fig.show()

In [15]:
df.dtypes

TICKER            object
PER               object
DATE      datetime64[ns]
TIME               int64
OPEN             float64
HIGH             float64
LOW              float64
CLOSE            float64
VOL                int64
dtype: object

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Импорт данных
data = df[['DATE', 'CLOSE']].copy()

# Преобразование данных к нужному формату и масштабирование
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[['CLOSE']])
data['CLOSE_scaled'] = data_scaled

# Определение длины последовательности для LSTM
sequence_length = 30  # Прогноз на неделю

# Формирование временных последовательностей для обучения LSTM
def create_sequences(data, seq_length):
    x, y = [], []
    for i in range(len(data)-seq_length):
        x.append(data[i:(i+seq_length)])
        y.append(data[i+seq_length])
    return np.array(x), np.array(y)

# Создание последовательностей
X, y = create_sequences(data['CLOSE_scaled'].values, sequence_length)

# Разделение данных на обучающий и тестовый наборы
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Построение модели LSTM
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(sequence_length, 1)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Цикл обучения модели с учётом предсказанных дней
for i in range(sequence_length):
    # Обучение модели
    model.fit(X_train.reshape((X_train.shape[0], X_train.shape[1], 1)), y_train, epochs=100, batch_size=8, verbose=1)

    # Предсказание
    future_data = np.array(data_scaled[-sequence_length:]).reshape((1, sequence_length, 1))
    predicted_scaled = model.predict(future_data)
    predicted_value = scaler.inverse_transform(predicted_scaled)

    # Добавление новой строки для предсказания
    last_date = pd.to_datetime(data['DATE'].max()) + pd.DateOffset(days=1)
    data = data.append({'DATE': last_date, 'CLOSE': np.nan}, ignore_index=True)

    # Замена NaN на предсказанное значение
    data.loc[data.index.max(), 'CLOSE'] = predicted_value[0][0]

    # Обновление данных для обучения
    data_scaled = scaler.transform(data[['CLOSE']])
    X, y = create_sequences(data_scaled, sequence_length)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    print(f'{i} точка данных получена')

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.plot(data['DATE'][-sequence_length:], data['CLOSE'][-sequence_length:], label='Прогноз на неделю', marker='o', color='red')
plt.plot(data['DATE'][:-sequence_length], data['CLOSE'][:-sequence_length], label='Исходные данные', marker='o', color='blue')
plt.xlabel('Дата')
plt.ylabel('CLOSE')
plt.title('Прогноз с использованием LSTM')
plt.legend()
plt.show()


ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
# Визуализируем данные с использованием Plotly Express
fig = px.line(data, x='DATE', y='CLOSE', title='Прогноз цен закрытия акций')
fig.update_xaxes(title_text='Дата')
fig.update_yaxes(title_text='Цена закрытия')

# Выделяем исходные данные 2022-2023 годов красным цветом, предсказанные данные 2024 - зелёным
fig.add_scatter(x=data['DATE'][:-sequence_length], y=data['CLOSE'][:-sequence_length], mode='lines+markers', name='Исходные данные', line=dict(color='red'))
fig.add_scatter(x=data['DATE'][-sequence_length:], y=data['CLOSE'][-sequence_length:], mode='lines+markers', name='Прогноз', line=dict(color='green'))

# Отобразим график
fig.show()